# Project 2: Read Alignment and Genome Assembly with Dynamic Programming

## Module 3 Introduction

Dynamic Programming Algorithms 
- flexible
- allow us to look for approximate matches considering insertions/deletions 
- find similar substrings between strings 

And then we move onto genome assembly without a reference genome
- begin to discuss concepts related to that. 

## Solving Edit Distance Problem 

Two different ways to measure difference betwen two strings 

Hamming distance = minimum # of substitutions needed to turn one string to another (both strings are of the same length). 
- pretty easy to find the Hamming Distance 

Edit distance = mimumum # of substitutions/insertions/deletions needed to turn one string to another (both strings do not have to be the same length). 
- pretty hard to find the edit distance

Have to approach edit distance with a couple steps: 

If we have strings, x and y, of the same length, we can say the following about the hamming_distance and edit_distance between x and y 

1. edit_distance <= hamming_distance
- in the case there are only substitution differences, they'll be equal
- if there's an indel, edit_distance will be less because you don't have to make all those substituions, you can just shift it. 

If we have strings, x and y, of different length, we can say the following about the edit_distance between x and y 

1. edit_distance >= abs(len(x)-len(y))
- we always have to fill the gap in lengths. so if there are no substitutions, adding those characters will be the = edit_distance 
- if there are substitutions, adding those characters will be > edit distance. 

Algorithm to correct the edit distance: 

- Take two very long strings
- Consider the prefix of the strings (everything but the last base)
- C and A are more generally represented as X and Y, differing bases. 

Fig. 1

- Take the edit distance between the two prefixes 

- The mimumum of these three numbers is the edit distance between the two strings: 

delta = 0 if x = y or 1 otherwise.
ed(changing alpha to beta) + (delta)
ed(changing alpha(x) to beta) + (adding one base to beta)
ed(changing alpha to ( beta(y)) ) + (adding one base to alpha)

Fig. 2 

- all of this can be represented by a recursive function (below)
problem : recursion is slow. super slow.
solution: use dynamic programming. 

Fig. 1 
<img src="/Users/arshmeetkaur/Genomic_Data_Science/course3_algorithms/images/prefixes.png" alt="Local Image" width="300">

Fig. 2
<img src="/Users/arshmeetkaur/Genomic_Data_Science/course 3 /images/edit_distance_formula.png" alt="Local Image" width="300">

In [1]:
def hamming_distance(x, y): 
    """ returns hamming distance (subs only). """
    subs = 0
    for i in range(len(x)):
        if x[i] != y[i]: 
            subs += 1 
    return subs 

In [12]:
def edit_distance_recursion(a, b): 
    """ returns edit distance recursively (slow). """

    # base case - one empty string means edit distance between them is equal to the other string. 
    if len(a) == 0: 
        return len(b) 
    if len(b) == 0:
        return len(a)

    delt = 1 if a[-1] != b[-1] else 0
    return min(edit_distance_recursion(a[:-1], b[:-1]) + delt, 
               edit_distance_recursion(a, b[:-1]) + 1, 
               edit_distance_recursion(a[:-1], b) + 1)

## Using dynamic programming for edit distance

- need to rewrite the recursive function 
- characters of string x label rows, characters of string y label columns (fig 1)
- first row and first column are labeled with epsilon, for the empty string
- fill in each position in the matrix with the edit distance
- the total edit distance is in the bottom right corner (fig 2)

How this works: 
- initialize 1rst row to 0... len(x), initialize 1rst column to 0... len(y), this makes sense if you think about the edit distance. 
- think of each of the edit distance terms as one position in the matrix (see fig 3). the edist( alpha(x) + beta(y) ) = min (three other cells)

- this dynamic programming approach is good for a lot of things. 

Fig 1 
<img src="/Users/arshmeetkaur/Genomic_Data_Science/course 3 /images/matrix.png" alt="Local Image" width="500">

Fig 2
<img src="/Users/arshmeetkaur/Genomic_Data_Science/course 3 /images/corner_matrix.png" alt="Local Image" width="500">

Fig 3 
<img src="/Users/arshmeetkaur/Genomic_Data_Science/course 3 /images/boxes.jpeg" alt="Local Image" width="500">
<img src="/Users/arshmeetkaur/Genomic_Data_Science/course 3 /images/fill_in.png" alt="Local Image" width="500">
<img src="/Users/arshmeetkaur/Genomic_Data_Science/course 3 /images/final_matrix.png" alt="Local Image" width="500">

## Practical: Implementing dynamic programming for edit distance

In [3]:
def edit_distance(x, y):
    """ uses dynamic programming to compute edit distance. """
    D = [] # intial array 
    
    # initialize the array as rows = (x+1) by cols = (y+1) (+1 because we're counting the empty prefix)
    for i in range(len(x) + 1):
        D.append([0] * (len(y) + 1))

    # initialize first row as 0 through y+1
    for i in range(len(y) + 1):
        D[0][i] = i
    # initialize first column as 0 through x+1 
    for i in range(len(x) + 1):
        D[i][0] = i
    
    # step through and fill in the rest of the matrix (not going through the top left corner)
    # for each row... 
    for r in range(1, len(x) + 1): 
        # for each column...
        for c in range(1, len(y) + 1): 
            # check if the strings are equal at the indices corresponding to each character's row/column 
            delt = 0 if x[r-1] == y[c-1] else 1
            # evaluate the values of the top, left and diagonal choices
            top = D[r-1][c] + 1
            left = D[r][c-1] + 1
            diagonal = D[r-1][c-1] + delt 
            # new spot filled by min of all choices 
            D[r][c] = min(top, left, diagonal)

    # once done filling in the matrix, return bottom right corner value 
    return D[len(x)][len(y)]


In [15]:
%%time 
x = 'shake spea'
y = 'Shakespear'
print(edit_distance(x,y))

3
CPU times: user 95 μs, sys: 4 μs, total: 99 μs
Wall time: 116 μs


In [16]:
%%time 
print(edit_distance_recursion(x,y)) # max recursion depth exceeded.

3
CPU times: user 1.4 s, sys: 4.73 ms, total: 1.41 s
Wall time: 1.42 s


## Lecture: A new solution to approximate matching

Fig 1 - initial matrix of P and T 

<img src="/Users/arshmeetkaur/Genomic_Data_Science/course 3 /images/starting_pt.png" alt="Local Image" width="500">

Fig 2 - initialize values of first row and column 

<img src="/Users/arshmeetkaur/Genomic_Data_Science/course 3 /images/initialize_rc_pt.png" alt="Local Image" width="500">

- we'll create a matrix of P rows and T columns (fig 1)
- we will initialize values of the first column to be 0 to len(p)
- we will initialize values of the first row to all be 0. 
Why? Because that first index of P could be at any offset of T, we don't know yet. (fig 2)

Fig 3 - fill in matrix same way, get index of best match, find path from best match to top row. 

<img src="/Users/arshmeetkaur/Genomic_Data_Science/course 3 /images/match_offset.png" alt="Local Image" width="500">

- fill in the matrix in exactly the same way. 

- to identify approx matches: identify the occurence with the lowest edit distance. look in the len(p) row and find the smallest value (smallest edit distance)

- to get the index of the best match work backwards. Look at the top bottom and diagonal cells, find the smallest value and go backwards to the min value. At the end, you will end up somewhere in the top row.

- look at the column we're in, that's the offset of T that matches with P

Fig 4 - The path tells us a bunch of stuff 

<img src="/Users/arshmeetkaur/Genomic_Data_Science/course 3 /images/match_offset_pattern.jpeg" alt="Local Image" width="500">

- big problem = quite slow. O( (len(p)) * len(t) ) because you have to go through the entire matrix. 
- impractical to use on your own
- usually, this technique is combined with others.
- by combining this with previous techniques (index/pigeonhole principle), we can have the speed of the previous techniques and the approximate matching which handles gaps and indels

## Meet the family: global and local alignment 

Variations on theme of dynamic programming for edit distance for global and local alignment problems 

- edit distance penalizes in and del and subs the same. 
no difference between insertion and deletion 
no difference between substitution of A/T and A/G

- in reality, it might make more sense to penalize less frequent types of mutations more. 

Global Alignment: 

Fig 1 - Categories of DNA mutations 
<img src="/Users/arshmeetkaur/Genomic_Data_Science/course 3 /images/purines_pyrm.png" alt="Local Image" width="500">

A and G are purines, C and T are pyrimidines. 
- going across types is transversion 
- switching a purine for a purine or pyrimidine for pyrimidine, those are transitions. 
- in reality, transitions were twice as frequent as tranversions. 
- want to penalize transversions more than transitions. 

Fig 2 - rates of subs and indels 
<img src="/Users/arshmeetkaur/Genomic_Data_Science/course 3 /images/sub_indel_rates.png" alt="Local Image" width="500">
'
- indels less frequent than subsitutions, so penalize them more.

Fig 3- penalty matrix 
<img src="/Users/arshmeetkaur/Genomic_Data_Science/course 3 /images/penalty_matrix.png" alt="Local Image" width="500">

- depends on which species you're working with, this is for humans.

Step: Initialize first row and column 
- top right char is always 0
- see the practical code

Fig 4 - editing the dynamic programming algo. 
<img src="/Users/arshmeetkaur/Genomic_Data_Science/course 3 /images/penalty_function.png" alt="Local Image" width="500">

- all that has to change is what we add to each edit distance. now we look up what value to add in a penalty matrix function 
- other than that, the global alignmnet algorithm is the exact same.

Local Alignment: 

Fig 1 - Local Alignment - find the most similar pair of substrings between strings X and Y. 
<img src="/Users/arshmeetkaur/Genomic_Data_Science/course 3 /images/local_alignment.png" alt="Local Image" width="500">

- very difficult because we have to consider all possible pairs of substrings between X and Y: which is like len(p)^2 * len(t)^2 pairs 

Fig 2 - Scoring Matrix 
<img src="/Users/arshmeetkaur/Genomic_Data_Science/course 3 /images/scoring_matrix.png" alt="Local Image" width="500">

- initialize 0s and 0s for row and column
- fill in with the scoring matrix function as the addition 
(use a scoring matrix: give a + bonus for match and - deduction for each mismatch.)
 
Fig 3 - Dynamic Programming Matrix 
<img src="/Users/arshmeetkaur/Genomic_Data_Science/course 3 /images/la_matrix.png" alt="Local Image" width="500">

- many elements are 0, because the goal of local laignment is to find parts of X and Y that match close enough to highlight the reigons of max
- to get the indices: 
- find the biggest number (anywhere in the matrix) and traceback, stop when you reach and element = 0. 
- the substrings are the values along x and y across the space that the diagonal is covering. 
- you can tell shape of the substring match by the traceback path.

## Practical: Implementing global alignment

In [20]:
# global alignment 

# A and G are purines
# C and T are pyrimidines
alphabet = ['A', 'C', 'G', 'T']
# penalty matrix 
# nucleotide against A, C, G, T, skip
# row A, C, G, T
penalty = [ [0, 4, 2, 4, 8], #A 
            [4, 0, 4, 2, 8], #C
            [2, 4, 0, 4, 8], #G
            [4, 2, 4, 0, 8], #T 
            [8, 8, 8, 8, 8] ] #all skips

def global_alignment(x, y):
    D = [] # intial array 
    
    # initialize the array as rows = (x+1) by cols = (y+1) (+1 because we're counting the empty prefix)
    for i in range(len(x) + 1):
        D.append([0] * (len(y) + 1))

    """ 
    Step: Initialize first row and column 
    - top right char is always 0
    - the first column should contain all skips, the penalties for skipping a character.
    - for each column value in the first row (after column 1), penalties for skipping a character. 
    """
    
    # initialize first column 
    # only 1, len(x) + 1 because top corner should be 0. e
    for i in range(1, len(x) + 1):
        D[i][0] = D[i-1][0] + penalty[alphabet.index(x[i-1])][-1]
     # initialize first row 
    for i in range(1, len(y) + 1):
        # take the character to the current character and add the penalty of a skip for that character
        D[0][i] = D[0][i-1] + penalty[-1][alphabet.index(y[i-1])]
    
    # step through and fill in the rest of the matrix (not going through the top left corner)
    # for each row... 
    for r in range(1, len(x) + 1): 
        # for each column...
        for c in range(1, len(y) + 1): 
            # evaluate the values of the top, left and diagonal choices
            top = D[r-1][c] +  penalty[alphabet.index(x[r-1])][-1] # penalty for skipping a character in y 
            left = D[r][c-1] + penalty[-1][alphabet.index(y[c-1])] # penalty for skipping a character
            diagonal = D[r-1][c-1] + penalty[alphabet.index(x[r-1])][alphabet.index(y[c-1])] # penalty for difference in char
            # new spot filled by min of all choices 
            D[r][c] = min(top, left, diagonal)

    # once done filling in the matrix, return bottom right corner value 
    return D[len(x)][len(y)]

In [21]:
x = 'TACCAGATTCGA'
y = 'TACCACATTCGA'
global_alignment(x, y)

4

## Lecture: Read alignment in the field

How does read alignmnet actually happen in the field? 

- indexing + dynamic programming are a powerful combination 

Fig 1: Indexing. 
<img src="/Users/arshmeetkaur/Genomic_Data_Science/course 3 /images/indexing.png" alt="Local Image" width="500">

- pros: fast, narrow down a small number of potential matches within a huge genome. 
- cons: bad at flexibility, doing approximate matches/gaps. 

Fig 2: Dynamic Programming

<img src="/Users/arshmeetkaur/Genomic_Data_Science/course 3 /images/dynamic_programming.png" alt="Local Image" width="500">

- pros: naturally good at being flexible, doing approximate matches even with indels. 
- cons: really slow. Huge table sizes for a P(read) by T(reference genome) table. Would take years to align one genome sequence. 

Fig 3: work together
<img src="/Users/arshmeetkaur/Genomic_Data_Science/course 3 /images/combined.png" alt="Local Image" width="500">

- So they have to work together to combine these 

## Assembly Problem: Working from Scratch

- de novo shotgun assembly: putting together a puzzle with no completed puzzle. 
    - species that has never been studied before and we dont have a reference for like the human genome project. 

- fundamentally difficult problem, more difficult than read alignment. 

## Lecture: First and second laws of assembly

Fig 1: The assembly problem 
<img src="/Users/arshmeetkaur/Genomic_Data_Science/course 3 /images/assembly_problem.png" alt="Local Image" width="500">

We are given reads from a genome that we would like to assemble. 
- we dont know where the reads came from. 
- we also don't know what the genome is. 

Vocabulary: 

Fig 2- coverage 
<img src="/Users/arshmeetkaur/Genomic_Data_Science/course 3 /images/coverage.png" alt="Local Image" width="500">

- coverage = the reads per each position. The reads are giving us (coverage) number of votes on one position. 
    - sometimes, not all reads agree on what base should be there. 

Fig 3 - average coverage 
<img src="/Users/arshmeetkaur/Genomic_Data_Science/course 3 /images/average_coverage.png" alt="Local Image" width="500">

- average coverage = (total length of all reads)/(total length of the genome)

How do you peice together the genome? 

Fig 4- First law of assembly 
<img src="/Users/arshmeetkaur/Genomic_Data_Science/course 3 /images/first_law.png" alt="Local Image" width="500">

- if two reads come from the same genome and the suffix of one read is very similar to the prefix of another, they are likely to overlap in the genome. 

Fig 5- sequencing erors, differences in overlapping reads because of polyploidy
<img src="/Users/arshmeetkaur/Genomic_Data_Science/course 3 /images/differences.png" alt="Local Image" width="500">

- ^^ reasons why two overlapping reads have a couple mismatches. 

Fig 6 - Second law of assembly 

<img src="/Users/arshmeetkaur/Genomic_Data_Science/course 3 /images/second_law.png" alt="Local Image" width="500">
- more coverage (more reads per base of the genome), more longer overlaps 

## Overlap Graphs

- talked about overlaps in reads that help us glue the genome together
- we'll talk about a way to put all the overalps into one structure 

Fig 1 and 2 - Directed graph:
<img src="/Users/arshmeetkaur/Genomic_Data_Science/course 3 /images/hamlet.png" alt="Local Image" width="500">
<img src="/Users/arshmeetkaur/Genomic_Data_Science/course 3 /images/node_edge.png" alt="Local Image" width="500">
- node 
- edge 
- nodes and edges have some meaning
- in a directed graph, the edges have a direction. 
- in this case, nodes are character, edges are killing. 

Fig 3 - read overlap graphs
<img src="/Users/arshmeetkaur/Genomic_Data_Science/course 3 /images/read_overlap.png" alt="Local Image" width="500">
- draw an edge from the read that has a suffix that overlaps with the other node's prefix
- each edge is labeled with the amount that overlaps: has to be an exact match over 4

Fig 5- How to get the genome from this graph
<img src="/Users/arshmeetkaur/Genomic_Data_Science/course 3 /images/find_genome.png" alt="Local Image" width="500">
- the orignal genome is going to be obtained by walking through one path in this graph. 
- follow the path of the biggest edge value throughout the tree
- in reality, much less idealized

## Practical: Overlaps between pairs of reads

In [26]:
def overlap(a, b, min_length = 3): # min length of the overlap. 
    """ find overlap betwedn suffix of a and prefix of b. 
    Mimumum overlap must be 3 between the suffix and prefix by default.
    If the suffix and prefix overlap by less than min_length, return 0. 
    If the suffix and prefix overlap by more than min_length, return their max overlap length. """
    start = 0 # start seaching at index 0
    while True:
        # search for the first prefix substring of b (length 3) in a 
        start = a.find(b[:min_length], start) 
        # if there was no occurence of an overlap of size min_length, return 0. 
        if start == -1:
            return 0 
        # if the start is not -1, we found prefix of b in a. 
        # check that the prefix of a matches with the suffix of a (after the start occurence of b)
        if b.startswith(a[start:]):
            return len(a) - start 
        # if the prefix of b didn't match fully with the suffix of a after start, 
        # increment start so we can start searching at the next place. 
        start += 1 

In [32]:
overlap('TTACGTG', 'CGTGTGC')

4

In [33]:
overlap('TTACGT', 'CGTGTGC')

3

In [31]:
overlap('TTACGT', 'GTGTGC')

0

## Practical: Finding and representing all overlaps

In [3]:
from itertools import permutations

list(permutations([1,2,3], 3)) # gives you all possible permutations of the list. 

[(1, 2, 3), (1, 3, 2), (2, 1, 3), (2, 3, 1), (3, 1, 2), (3, 2, 1)]

In [39]:
def naive_overlap_map(reads, k): 
    """ Reads and min overlap length k """
    olaps = {} # dictionary of overlaps 

    """ For all pairs of reads in a set of reads, 
    calculate the overlap between a and b (with min_length = k)
    if the overlap is greater than zero, record that overlap in the dictionary. 
    """
    for a, b in permutations(reads, 2):
        olen = overlap(a, b, min_length=k)
        if olen > 0: 
            olaps[(a, b)] = olen 
    
    return olaps 

In [42]:
reads = ['ACGGATC', 'GATCAAGT', 'TTCACGGA']
print(naive_overlap_map(reads, 3))

{('ACGGATC', 'GATCAAGT'): 4, ('TTCACGGA', 'ACGGATC'): 5}
